In [ ]:
# coding=UTF-8
import os
import sys
import time
import gc

import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold

import config

from sklearn.metrics import roc_auc_score
from DataReader import FeatureDictionary, DataParser
from DeepFM import DeepFM

# 419858
def _load_data():

#     dfTrain = pd.read_csv(config.TRAIN_FILE)
#     dfTest = pd.read_csv(config.TEST_FILE)
    dfTrain = pd.read_csv('../../Tencent Data/train_value2index.csv')
    dfTest = pd.read_csv('../../Tencent Data/test_value2index.csv')
    dfTrain.loc[dfTrain['label'] == -1, 'label'] = 0

#     delcol = ['ct','os', 'topic1', 'topic2', 'topic3', 'kw1', 'kw2', 'kw3',
#     'interest1', 'interest2', 'interest3', 'interest4', 'interest5', 
#     'marriageStatus', 'appIdAction', 'appIdInstall']
#     for each in delcol:
#         del dfTrain[each]
#         del dfTest[each]
        
#      加入交叉特征
#     cross_feat = pd.read_csv('../data_cross_feat.csv')
#     dfTrain = pd.concat([dfTrain, cross_feat[:8798814]])
#     dfTest = pd.concat([dfTest, cross_feat[8798814:]])
#     del cross_feat
#     gc.collect()
    
    dfTrain.fillna(419859,inplace=True)
    dfTest.fillna(419859,inplace=True)
    cols = [c for c in dfTrain.columns if c not in ["uid", "label"]]
    cols = [c for c in cols if (not c in config.IGNORE_COLS)]

    X_train = dfTrain[cols].values
    y_train = dfTrain["label"].values


    return dfTrain, dfTest, X_train, y_train


#def _run_base_model_dfm(dfTrain, dfTest, folds, dfm_params):
def _run_base_model_dfm(Xi_train ,Xv_train, y_train, Xi_test, Xv_test, folds, dfm_params):
#     fd = FeatureDictionary(dfTrain=dfTrain, dfTest=dfTest,
#                            numeric_cols=config.NUMERIC_COLS,
#                            #-------------------增加多值特征列
#                            multivalued_cols=config.MULTIVALUED_COLS,
                           
#                            ignore_cols=config.IGNORE_COLS)

#     data_parser = DataParser(feat_dict=fd)

#     Xi_train, Xv_train, y_train = data_parser.parse(df=dfTrain, has_label=True)

#     Xi_test, Xv_test, res_test = data_parser.parse(df=dfTest)
    
#     dfm_params["feature_size"] = fd.feat_dim
    dfm_params["feature_size"] = 419859
    dfm_params["field_size"] = 31
    
    y_train = dfTrain["label"].values.tolist()
    del dfTrain['label']#,dfTrain['uid']
    
    Xi_train = np.array(dfTrain).tolist()
    Xi_train_ = []
    for row in Xi_train:
        a = []
        for each in row:
            if isinstance(each,int):
                a.append(each)
            elif isinstance(each,float):
                a.append(int(each))
            elif isinstance(each,str):
                a.append(each.split())
            else:
                print each
        Xi_train_.append(a)
    Xi_train = Xi_train_
    del Xi_train_
    
    
    Xv_train = np.ones([len(Xi_train),len(Xi_train[0])])
    
    test1 = pd.read_csv('../../Tencent Data/test1.csv')
    res_test = test1[['aid','uid']]
    del test1
    Xi_test = np.array(dfTest).tolist()
    Xi_test_ = []
    for row in Xi_test:
        a = []
        for each in row:
            if isinstance(each,int):
                a.append(each)
            elif isinstance(each,float):
                a.append(int(each))
            elif isinstance(each,str):
                a.append(each.split())
            else:
                print each
        Xi_test_.append(a)
    Xi_test = Xi_test_
    del Xi_test_
    Xv_test = np.ones([len(Xi_test),len(Xi_test[0])])
    

    
    y_train_meta = np.zeros((dfTrain.shape[0], 1), dtype=float)

    y_test_meta = np.zeros((dfTest.shape[0], 1), dtype=float)

    _get = lambda x, l: [x[i] for i in l]

    auc_results_cv = np.zeros(len(folds), dtype=float)

    auc_results_epoch_train = np.zeros((len(folds), dfm_params["epoch"]), dtype=float)

    auc_results_epoch_valid = np.zeros((len(folds), dfm_params["epoch"]), dtype=float)

    for i, (train_idx, valid_idx) in enumerate(folds):
        Xi_train_, Xv_train_, y_train_ = _get(Xi_train, train_idx), _get(Xv_train, train_idx), _get(y_train, train_idx)
        Xi_valid_, Xv_valid_, y_valid_ = _get(Xi_train, valid_idx), _get(Xv_train, valid_idx), _get(y_train, valid_idx)

        dfm = DeepFM(**dfm_params)

        dfm.fit(Xi_train_, Xv_train_, y_train_, Xi_valid_, Xv_valid_, y_valid_)

        y_train_meta[valid_idx,0] = dfm.predict(Xi_valid_, Xv_valid_)

        y_test_meta[:,0] += dfm.predict(Xi_test, Xv_test)

        auc_results_cv[i] = roc_auc_score(y_valid_, y_train_meta[valid_idx])
        auc_results_epoch_train[i] = dfm.train_result
        auc_results_epoch_valid[i] = dfm.valid_result
        
        gc.collect()

    y_test_meta /= float(len(folds))

    # save result
    if dfm_params["use_fm"] and dfm_params["use_deep"]:
        clf_str = "DeepFM"
    elif dfm_params["use_fm"]:
        clf_str = "FM"
    elif dfm_params["use_deep"]:
        clf_str = "DNN"
    print("%s: %.5f (%.5f)"%(clf_str, auc_results_cv.mean(), auc_results_cv.std()))
    filename = "%s_Mean%.5f_Std%.5f.csv"%(clf_str, auc_results_cv.mean(), auc_results_cv.std())
    _make_submission(res_test, y_test_meta, filename)

    _plot_fig(auc_results_epoch_train, auc_results_epoch_valid, clf_str)
    
    return y_train_meta, y_test_meta


def _make_submission(res, y_pred, filename="submission.csv"):
    res['score'] = y_pred.flatten()
    save_path = os.path.join(config.SUB_DIR, filename)
    res.to_csv(save_path, index=False, float_format="%.6f")
    os.system('zip %s.zip %s' % (save_path, save_path))

def _plot_fig(train_results, valid_results, model_name):
    colors = ["red", "blue", "green"]
    xs = np.arange(1, train_results.shape[1]+1)
    plt.figure()
    legends = []
    for i in range(train_results.shape[0]):
        plt.plot(xs, train_results[i], color=colors[i], linestyle="solid", marker="o")
        plt.plot(xs, valid_results[i], color=colors[i], linestyle="dashed", marker="o")
        legends.append("train-%d"%(i+1))
        legends.append("valid-%d"%(i+1))
    plt.xlabel("Epoch")
    plt.ylabel("AUC")
    plt.title("%s"%model_name)
    plt.legend(legends)
    T = int(time.time())
    plt.savefig("./fig/%s_%d.png"%(model_name,T))
    plt.close()

In [ ]:
# load data
dfTrain, dfTest, X_train, y_train = _load_data()

# folds
folds = list(StratifiedKFold(n_splits=config.NUM_SPLITS, shuffle=True,
                             random_state=config.RANDOM_SEED).split(X_train, y_train))
del X_train,y_train
gc.collect()

In [ ]:
# params
dfm_params = {
    "use_fm": True,
    "use_deep": True,
    "embedding_size": 8,
    "dropout_fm": [1.0, 1.0],
    "deep_layers": [100, 100],
    "dropout_deep": [0.9, 0.9, 0.9],
    "deep_layers_activation": tf.nn.relu,
    "epoch": 15,
    "batch_size": 2048,
    "learning_rate": 0.001,
    "optimizer_type": "adam",
    "batch_norm": 1,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.01,
    "verbose": True,
    #"eval_metric": gini_norm,
    "random_seed": config.RANDOM_SEED
}

In [ ]:
# ------------------ DeepFM Model ------------------
y_train_dfm, y_test_dfm = _run_base_model_dfm(dfTrain, dfTest, folds, dfm_params)

In [ ]:
# ------------------ FM Model ------------------
fm_params = dfm_params.copy()
fm_params["use_deep"] = False
y_train_fm, y_test_fm = _run_base_model_dfm(dfTrain, dfTest, folds, fm_params)

In [ ]:
# ------------------ DNN Model ------------------
dnn_params = dfm_params.copy()
dnn_params["use_fm"] = False
y_train_dnn, y_test_dnn = _run_base_model_dfm(dfTrain, dfTest, folds, dnn_params)

# 以下为修改版，支持多值特征
# 多值特征格式为Xi = [1,2,3,4,[5,6,7]]
## 其中[5，6，7]为多值特征

In [1]:
import os
import sys
import time
import gc

import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt


import config

from sklearn.metrics import roc_auc_score
from DeepFM import DeepFM
import random 


dfm_params = {
    "use_fm": True,
    "use_deep": True,
    "embedding_size": 10,
    "dropout_fm": [1.0, 1.0],
    "deep_layers": [300, 300, 300],
    "dropout_deep": [1, 1, 1, 1],
    "deep_layers_activation": tf.nn.relu,
    "epoch": 1,
    "batch_size": 4096,
    "learning_rate": 0.001,
    "optimizer_type": "adam",
    "loss_type": "logloss",
    "batch_norm": 1,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.01,
    "verbose": True,
    #"eval_metric": gini_norm,
    "random_seed": config.RANDOM_SEED
}


has_cross = False
valid_rate = 0.1

print('loading data...')
test1 = pd.read_csv('../../Tencent Data/test2.csv')
res_test = test1[['aid','uid']]
del test1
Y_train = pd.read_csv('../../Tencent Data/label.csv')
Y_train.loc[Y_train['label'] == -1,'label'] = 0
Y_train = Y_train.values.reshape(-1,)

data = pd.read_csv('../../Tencent Data/trans2.csv')
data = data.astype('str')

Test = data.iloc[8798814:]
Train = data.iloc[:8798814].values
del data



if has_cross:
    print('loading cross feat...')
    cross_feat = pd.read_csv('../../Tencent Data/cross_feat.csv')
    Xi_cross = None
    F = True
    row_num = len(cross_feat)
    for i in range(419860,419908):
        if F:
            Xi_cross = np.array([str(i)]).repeat(row_num).reshape(-1,1)
            F = False
        else:
            Xi_cross = np.hstack((Xi_cross,np.array([str(i)]).repeat(row_num).reshape(-1,1)))

    Xv_cross = cross_feat.values

    dfm_params["feature_size"] = 419763+len(cross_feat.columns)
    dfm_params["field_size"] = 31+len(cross_feat.columns)
    del cross_feat
    
    Xi_test = np.hstack((np.array(Test),Xi_cross[8798814:]))
    del Test
    Xv_test = np.hstack((np.ones([len(Xi_test),len(Xi_test[0])]),Xv_cross[8798814:]))
    
    Xv_train = np.hstack((np.ones([len(Train),len(Train[0])]),Xv_cross))
    Xi_Train = np.hstack((Train,Xi_cross))
else:
    dfm_params["feature_size"] = 419763
    dfm_params["field_size"] = 31
    
    Xi_test = np.array(Test)
    del Test
    Xv_test = np.ones([len(Xi_test),len(Xi_test[0])])
    
    Xv_train = np.ones([len(Train),len(Train[0])])
    Xi_train = Train

del Train

train_size = len(Xi_train)
chunksize = 7918933
dfm = DeepFM(**dfm_params)

valid_size = int(valid_rate * train_size)
shuffle_index = range(train_size)
random.shuffle(shuffle_index)

train_index = shuffle_index[valid_size:]
valid_index = shuffle_index[:valid_size]

print('split train to batch...')
batch_index = []
for i in range(len(train_index) / chunksize):
     batch_index.append(shuffle_index[i * chunksize:(i+1)*chunksize])
if (len(train_index) % chunksize) != 0:
    batch_index.append(shuffle_index[-(len(train_index) % chunksize):])

gc.collect()    

Xi_valid = Xi_train[valid_index]
Xv_valid = Xv_train[valid_index]
Y_valid = Y_train[valid_index]
print('Training begin...')
fff = open('output/log.txt','a')
fff.write('begin\n')
fff.close()
for i in range(len(batch_index)):
    print('Train No.%d batch, size is  %d...'% (i+1, len(batch_index[i])))
    xi_train = Xi_train[batch_index[i]]
    xv_train = Xv_train[batch_index[i]]
    y_train = Y_train[batch_index[i]].tolist()
    dfm.fit(xi_train, xv_train, y_train, Xi_valid, Xv_valid, Y_valid)
    gc.collect()


loading data...
#params: 4891635
split train to batch...
Training begin...
Train No.1 batch, size is  7918933...
[1] train-result=0.7505, valid-result=0.7536 [1655.4 s]


In [16]:
# predict the whole train set     
pre = dfm.predict(Xi_train, Xv_train).tolist()
auc = roc_auc_score(Y_train, pre)
fff = open('output/log.txt','a')
fff.write('all train set testing\n')
fff.write('auc %f\ncomplete\n' % auc)
fff.close()
print(auc)

0.7520056875776785


In [ ]:
b = Xi_train.tolist()
a=[]
for i in range(173):
    a.append([])

for each in b:
    a[int(each[0])].append(b.index(each))

In [ ]:
a

In [17]:
# predict the test set and save the result to csv    
res_test['score'] = dfm.predict(Xi_test, Xv_test)
save_path = os.path.join(config.SUB_DIR, 'submission.csv')
res_test.to_csv(save_path, index=False, float_format="%.6f")
os.system('zip %s.zip %s' % (save_path, save_path))

0

In [4]:
dfm.saver.save(dfm.sess,'model2/test2')

'model2/test2'

In [ ]:
import tensorflow as tf

from DeepFM import DeepFM
dfm_params = {
    "use_fm": True,
    "use_deep": True,
    "embedding_size": 8,
    "dropout_fm": [1.0, 1.0],
    "deep_layers": [100, 100],
    "dropout_deep": [0.9, 0.9, 0.9],
    "deep_layers_activation": tf.nn.relu,
    "epoch": 15,
    "batch_size": 2048,
    "learning_rate": 0.001,
    "optimizer_type": "adam",
    "batch_norm": 1,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.01,
    "verbose": True,
    #"eval_metric": gini_norm,
    "random_seed": 2018
}


dfm_params["feature_size"] = 419859
dfm_params["field_size"] = 31
dfm = DeepFM(**dfm_params)

In [ ]:
import numpy as np
import tensorflow as tf
a= [['1','2','3','1 2 3','1 2 3'],['1','2','3','1 2 3','1 2 3'],['1','2','3','1 2 3','1 2 3']]
a=np.array(a).reshape(-1,)


split_tags = tf.string_split(a, " ")


tags =  tf.SparseTensor(
      indices=split_tags.indices,
      values=tf.string_to_number(split_tags.values, out_type=tf.int64, name=None), ## 这里给出了不同值通过表查到的index ##
      dense_shape=split_tags.dense_shape)

TAG_EMBEDDING_DIM = 3
embedding_params = tf.Variable(tf.truncated_normal([100, TAG_EMBEDDING_DIM]))


embedded_tags = tf.nn.embedding_lookup_sparse(embedding_params, sp_ids=tags, sp_weights=None)
embedded_tags = tf.reshape(embedded_tags,shape=[-1,3,3])

In [ ]:
with tf.Session() as s:
  s.run([tf.global_variables_initializer(), tf.tables_initializer()])
  print(s.run([feat_index[0]]))

In [ ]:
weights = tf.Variable(tf.random_uniform([100, 1], 0.0, 1.0), name="feature_bias")

In [ ]:
a= [['1','2','3','1 2 3','1 2 3'],['1','2','3','1 2 3','1 2 3']]


In [ ]:

feat_index = tf.convert_to_tensor(a)
feat_index = tf.reshape(feat_index,shape=[1,-1])